In [1]:
import sys
import os

sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path


['',
 '/opt/anaconda3/lib/python36.zip',
 '/opt/anaconda3/lib/python3.6',
 '/opt/anaconda3/lib/python3.6/lib-dynload',
 '/opt/anaconda3/lib/python3.6/site-packages',
 '/opt/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/desenfirman/.ipython',
 '/media/Data/desenfirman/development/som-clustering-knn-imputation']

In [2]:
from main_algorithm import self_organizing_maps as som
from main_algorithm import knn_imputation as knn

In [3]:
# Load dataset to jupyter notebook
from pandas import read_csv

def extract_dataset(file):
    dataset = read_csv(file)
    new_columns = dataset.columns.values
    new_columns[0] = 'First_Column'
    dataset.columns = new_columns
    dataset = dataset.set_index('First_Column')
    raw_dataset = dataset
    return raw_dataset

dataset = extract_dataset("../dataset_used/dataset_PMKS.csv")
dataset

dataset_used = dataset.iloc[:, :].values
dataset_knn_ed = knn.impute_dataset(dataset_used, 0)
dataset_norm = som.normalize_data(dataset_knn_ed)

attr_size = len(dataset_used[0])
weight = som.init_som_net(6, 6, attr_size)
trained_weight = som.training(dataset_norm, weight, 100, 1, 0.7)
som.silhouette(trained_weight, dataset_norm)


Silhouette data 0 Cluster  0;1; : -0.35391977769299865
Silhouette data 27 Cluster  0;1; : 0.15213618547155286
-------------------------------------------------------
Silhouette data 1 Cluster  0;0; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  4;2; : 0
-------------------------------------------------------
Silhouette data 3 Cluster  1;4; : -0.29936335628663463
Silhouette data 14 Cluster  1;4; : -0.05821182942106291
-------------------------------------------------------
Silhouette data 4 Cluster  2;1; : -0.026627717287604692
Silhouette data 16 Cluster  2;1; : -0.12233908289367659
Silhouette data 17 Cluster  2;1; : 0.19051717831146853
-------------------------------------------------------
Silhouette data 5 Cluster  3;1; : 0
-------------------------------------------------------
Silhouette data 6 Cluster  3;3; : 0
-------------------------------------------------------
Silhouette data 7 Cluster  1;1; : 0.27686013838509305
Silhouette data 18 Clu

{'0;0;': {1: 0},
 '0;1;': {0: -0.35391977769299865, 27: 0.15213618547155286},
 '0;2;': {26: -0.1499925067419035, 32: -0.19017176304530964},
 '0;3;': {29: 0.025380123206648796, 31: 0.38489999810299297},
 '0;4;': {23: 0},
 '0;5;': {20: 0},
 '1;0;': {12: 0},
 '1;1;': {7: 0.27686013838509305,
  18: 0.22090021716281968,
  19: 0.18343905326826793},
 '1;2;': {28: 0},
 '1;3;': {11: 0.667899040302237,
  25: 0.5814377190587376,
  30: 0.49344172211347415,
  33: 0.5572375981861303,
  34: 0.675621462494544,
  35: 0.7057337926049487,
  37: 0.6676091292788429},
 '1;4;': {3: -0.29936335628663463, 14: -0.05821182942106291},
 '1;5;': {24: 0},
 '2;0;': {15: 0},
 '2;1;': {4: -0.026627717287604692,
  16: -0.12233908289367659,
  17: 0.19051717831146853},
 '2;2;': {21: 0},
 '2;3;': {10: 0},
 '2;4;': {13: -0.14766414412823908, 36: 0.20863768984584638},
 '3;0;': {22: 0},
 '3;1;': {5: 0},
 '3;2;': {9: 0},
 '3;3;': {6: 0},
 '3;5;': {8: 0},
 '4;2;': {2: 0},
 'avg': 0.1221963386920046}

# Pengujian parameter nilai learning rate (alpha)

Nilai yang akan diujikan
- 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1

Variabel kontrol
- Nilai eta = 0.1
- Jumlah epoch = 200
- Jumlah neuron = 5x5
- Nilai K = 0

In [4]:
from IPython.display import clear_output
from time import sleep
from pandas import DataFrame as df

# value to be tested
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# parameter control
eta = 0.1
epoch = 200
jumlah_neuron = (5,5)
k = 0

n_test = 10

qes = dict()


dataset_used = dataset.iloc[:, :].values
dataset_knn_ed = knn.impute_dataset(dataset_used, k)
dataset_norm = som.normalize_data(dataset_knn_ed)

for alpha in alphas:
    sum_all = 0
    qe_n = dict()
    for n in range(n_test):
        attr_size = len(dataset_used[0])
        print("Alpha: {0}, Eta: {1}, Epoch: {2}, Jumlah Neuron:{3}, K: {4} Try: {5}".format(alpha, eta, epoch, jumlah_neuron, k, n+1), end='\n')
        weight = som.init_som_net(jumlah_neuron[0], jumlah_neuron[1], attr_size)
        trained_weight = som.training(dataset_norm, weight, epoch, alpha, eta)
        qe = som.average_silhouette(trained_weight, dataset_norm)
        print("\nQE: {0}".format(qe), end='\n', flush=True)
        qe_n[n + 1] = qe
        sum_all += qe
    qe_n['avg'] = sum_all / n_test
    qes[str(alpha)] = qe_n
    clear_output()

(df.from_dict(qes)).to_csv('silhouette_test/alpha_testing.csv')

# Pengujian parameter nilai signifikasi update bobot (eta)

Nilai yang akan diujikan
- 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1

Variabel kontrol
- Nilai alpha = 1
- Jumlah epoch = 200
- Jumlah neuron = 5x5
- Nilai K = 0

In [10]:
from IPython.display import clear_output
from time import sleep
from pandas import DataFrame as df

# value to be tested
etas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# parameter control
alpha = 0.8
epoch = 200
jumlah_neuron = (5,5)
k = 0

n_test = 10

qes = dict()


dataset_used = dataset.iloc[:, :].values
dataset_knn_ed = knn.impute_dataset(dataset_used, k)
dataset_norm = som.normalize_data(dataset_knn_ed)

for eta in etas:
    sum_all = 0
    qe_n = dict()
    for n in range(n_test):
        attr_size = len(dataset_used[0])
        print("Alpha: {0}, Eta: {1}, Epoch: {2}, Jumlah Neuron:{3}, K: {4} Try: {5}".format(alpha, eta, epoch, jumlah_neuron, k, n+1), end='\n')
        weight = som.init_som_net(jumlah_neuron[0], jumlah_neuron[1], attr_size)
        trained_weight = som.training(dataset_norm, weight, epoch, alpha, eta)
        qe = som.average_silhouette(trained_weight, dataset_norm)
        print("\nQE: {0}".format(qe), end='\n', flush=True)
        qe_n[n + 1] = qe
        sum_all += qe
    qe_n['avg'] = sum_all / n_test
    qes[str(eta)] = qe_n

(df.from_dict(qes)).to_csv('silhouette_test/eta_testing.csv')

Alpha: 0.8, Eta: 0.1, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 1
Silhouette data 0QE:0.89271266086479156983033821537 Cluster  1;4; : 0.33236338077480476
Silhouette data 3 Cluster  1;4; : 0.27981974396148096
Silhouette data 4 Cluster  1;4; : 0.3023070203976831
Silhouette data 5 Cluster  1;4; : 0.0031722250073055428
Silhouette data 6 Cluster  1;4; : 0.07992906503581389
Silhouette data 7 Cluster  1;4; : 0.38252390749241527
Silhouette data 10 Cluster  1;4; : 0.23014097323014301
Silhouette data 11 Cluster  1;4; : 0.4739183279145955
Silhouette data 12 Cluster  1;4; : 0.18121559825738678
Silhouette data 13 Cluster  1;4; : 0.3189882350430678
Silhouette data 14 Cluster  1;4; : 0.3020589902973757
Silhouette data 15 Cluster  1;4; : 0.24079542673562626
Silhouette data 16 Cluster  1;4; : 0.1946503061161801
Silhouette data 17 Cluster  1;4; : 0.1987246203981104
Silhouette data 18 Cluster  1;4; : 0.21174520669461203
Silhouette data 19 Cluster  1;4; : 0.3769245215368577
Silhouette data 20 Cluster  1

Alpha: 0.8, Eta: 0.1, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 5
Silhouette data 0 Cluster  4;2; : 0.3675918199690218
Silhouette data 1 Cluster  4;2; : 0.09106641765034457
Silhouette data 3 Cluster  4;2; : 0.26452965759769564
Silhouette data 4 Cluster  4;2; : 0.29195777188232586
Silhouette data 5 Cluster  4;2; : -0.0035144743036891976
Silhouette data 6 Cluster  4;2; : 0.0728913511688957
Silhouette data 7 Cluster  4;2; : 0.37019963757117613
Silhouette data 10 Cluster  4;2; : 0.2194548284889243
Silhouette data 11 Cluster  4;2; : 0.4581637568540137
Silhouette data 12 Cluster  4;2; : 0.17452932542309196
Silhouette data 13 Cluster  4;2; : 0.306562743179402
Silhouette data 14 Cluster  4;2; : 0.2903579284453785
Silhouette data 15 Cluster  4;2; : 0.23065550333112372
Silhouette data 16 Cluster  4;2; : 0.24730205494545768
Silhouette data 17 Cluster  4;2; : 0.20425531965461818
Silhouette data 18 Cluster  4;2; : 0.2008800911986188
Silhouette data 19 Cluster  4;2; : 0.364162757828315
Silhouette 

Silhouette data 18 Cluster  0;1; : -0.13262349091852632
Silhouette data 21 Cluster  0;1; : -0.0773339895273361
Silhouette data 22 Cluster  0;1; : 0.15146576938943795
-------------------------------------------------------
Silhouette data 8 Cluster  0;2; : 0
-------------------------------------------------------
Silhouette data 24 Cluster  3;1; : 0
-------------------------------------------------------

QE: 0.18107058967171455
Alpha: 0.8, Eta: 0.1, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 9
Silhouette data 0 Cluster  0;3; : 0.320814860757388
Silhouette data 3 Cluster  0;3; : 0.26628731719595405
Silhouette data 4 Cluster  0;3; : 0.3204781347750491
Silhouette data 5 Cluster  0;3; : -0.0004669180045412893
Silhouette data 6 Cluster  0;3; : 0.1281401178423662
Silhouette data 7 Cluster  0;3; : 0.3683942694550828
Silhouette data 9 Cluster  0;3; : -0.038914116055660926
Silhouette data 10 Cluster  0;3; : 0.22415207977351972
Silhouette data 11 Cluster  0;3; : 0.4633588659100936
Silhouette da

Alpha: 0.8, Eta: 0.2, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 3
Silhouette data 0 Cluster  0;2; : 0.32701657024222436
Silhouette data 3 Cluster  0;2; : 0.24569453473650452
Silhouette data 4 Cluster  0;2; : -0.036100041154539464
Silhouette data 7 Cluster  0;2; : 0.20029443146590137
Silhouette data 9 Cluster  0;2; : -0.08015018116875401
Silhouette data 10 Cluster  0;2; : 0.14913270244044227
Silhouette data 11 Cluster  0;2; : 0.4514600594411493
Silhouette data 12 Cluster  0;2; : 0.026428220042842506
Silhouette data 13 Cluster  0;2; : 0.1470794123858826
Silhouette data 14 Cluster  0;2; : 0.12552476996163023
Silhouette data 15 Cluster  0;2; : -0.18216774965441626
Silhouette data 16 Cluster  0;2; : -0.13524993360533583
Silhouette data 17 Cluster  0;2; : -0.15071912542501442
Silhouette data 18 Cluster  0;2; : 0.13053434825505666
Silhouette data 19 Cluster  0;2; : 0.21944822235769249
Silhouette data 20 Cluster  0;2; : 0.04597992790400802
Silhouette data 21 Cluster  0;2; : -0.09697455166519

Alpha: 0.8, Eta: 0.2, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 7
Silhouette data 0 Cluster  4;2; : 0.320814860757388
Silhouette data 3 Cluster  4;2; : 0.26628731719595405
Silhouette data 4 Cluster  4;2; : 0.3204781347750491
Silhouette data 5 Cluster  4;2; : -0.0004669180045412893
Silhouette data 6 Cluster  4;2; : 0.1281401178423662
Silhouette data 7 Cluster  4;2; : 0.3683942694550828
Silhouette data 9 Cluster  4;2; : -0.038914116055660926
Silhouette data 10 Cluster  4;2; : 0.22415207977351972
Silhouette data 11 Cluster  4;2; : 0.4633588659100936
Silhouette data 12 Cluster  4;2; : 0.17726237411506496
Silhouette data 13 Cluster  4;2; : 0.3090824069438604
Silhouette data 14 Cluster  4;2; : 0.2937588825073186
Silhouette data 15 Cluster  4;2; : 0.18178602258448048
Silhouette data 16 Cluster  4;2; : 0.1856417641414912
Silhouette data 17 Cluster  4;2; : 0.19355571982857492
Silhouette data 18 Cluster  4;2; : 0.1989015853399959
Silhouette data 19 Cluster  4;2; : 0.36310760598460695
Silhouett

...

Alpha: 0.8, Eta: 0.9, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 9
Silhouette data 0 Cluster  0;2; : -0.08146129475457109
Silhouette data 20 Cluster  0;2; : -0.05171516472834211
Silhouette data 26 Cluster  0;2; : -0.15210131578101643
-------------------------------------------------------
Silhouette data 1 Cluster  3;1; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  2;0; : -0.07854408143319788
Silhouette data 9 Cluster  2;0; : -0.12127896240731972
-------------------------------------------------------
Silhouette data 3 Cluster  2;3; : -0.10520408692899105
Silhouette data 13 Cluster  2;3; : -0.22280152019918598
-------------------------------------------------------
Silhouette data 4 Cluster  3;3; : 0.09370064286662648
Silhouette data 17 Cluster  3;3; : 0.23985051021029188
-------------------------------------------------------
Silhouette data 5 Cluster  4;3; : 0
-------------------------------------------------------
Silhouette data 6 Cluster  

Alpha: 0.8, Eta: 1, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 2
Silhouette data 0 Cluster  2;0; : 0.4315345054355205
Silhouette data 26 Cluster  2;0; : 0.34310493256501773
-------------------------------------------------------
Silhouette data 1 Cluster  0;4; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  0;0; : -0.11626612589061996
Silhouette data 24 Cluster  0;0; : -0.13185959148180104
-------------------------------------------------------
Silhouette data 3 Cluster  4;2; : 0.14982620478561648
Silhouette data 7 Cluster  4;2; : 0.23435795570348197
Silhouette data 18 Cluster  4;2; : 0.12536580999957575
Silhouette data 19 Cluster  4;2; : 0.1654118874273662
-------------------------------------------------------
Silhouette data 4 Cluster  1;3; : 0.0752553061110037
Silhouette data 17 Cluster  1;3; : 0.12397873605719303
-------------------------------------------------------
Silhouette data 5 Cluster  4;4; : -0.09697685926398614
Silhouette data 8 C

Alpha: 0.8, Eta: 1, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 5
Silhouette data 0 Cluster  1;0; : 0.017469901415200217
Silhouette data 26 Cluster  1;0; : -0.19643894191958172
Silhouette data 27 Cluster  1;0; : 0.1275510523355271
-------------------------------------------------------
Silhouette data 1 Cluster  3;4; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  4;0; : -0.15826851605516865
Silhouette data 9 Cluster  4;0; : -0.10279613229609996
-------------------------------------------------------
Silhouette data 3 Cluster  0;3; : 0
-------------------------------------------------------
Silhouette data 4 Cluster  3;1; : 0.026627717287604692
Silhouette data 21 Cluster  3;1; : -0.2775683836827879
-------------------------------------------------------
Silhouette data 5 Cluster  3;0; : -0.132315854628615
Silhouette data 6 Cluster  3;0; : -0.038852332318049514
Silhouette data 22 Cluster  3;0; : -0.2889180478572761
---------------------------------

Alpha: 0.8, Eta: 1, Epoch: 200, Jumlah Neuron:(5, 5), K: 0 Try: 8
Silhouette data 0 Cluster  4;4; : 0.4315345054355205
Silhouette data 26 Cluster  4;4; : 0.34310493256501773
-------------------------------------------------------
Silhouette data 1 Cluster  1;4; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  0;4; : -0.11840629874418325
Silhouette data 9 Cluster  0;4; : -0.08514540265668331
-------------------------------------------------------
Silhouette data 3 Cluster  2;1; : 0
-------------------------------------------------------
Silhouette data 4 Cluster  2;4; : 0.17433488115502443
Silhouette data 17 Cluster  2;4; : 0.24003122323639534
-------------------------------------------------------
Silhouette data 5 Cluster  0;0; : -0.32142046650274275
Silhouette data 8 Cluster  0;0; : 0.14183083760793097
-------------------------------------------------------
Silhouette data 6 Cluster  1;0; : 0
------------------------------------------------------

-------------------------------------------------------
Silhouette data 23 Cluster  1;0; : 0
-------------------------------------------------------
Silhouette data 26 Cluster  0;2; : -0.3297573066714841
Silhouette data 28 Cluster  0;2; : 0.03661097906995166
-------------------------------------------------------
Silhouette data 29 Cluster  2;0; : 0.025380123206648796
Silhouette data 31 Cluster  2;0; : 0.38489999810299297
-------------------------------------------------------
Silhouette data 32 Cluster  0;1; : 0
-------------------------------------------------------

QE: 0.11541665242432839


# Pengujian parameter jumlah epoch

Nilai yang akan diujikan
- 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200

Variabel kontrol
- Nilai alpha = 0.5
- Nilai eta = 0.5
- Jumlah neuron = 5x5
- Nilai K = 0

In [20]:
from IPython.display import clear_output
from time import sleep
from pandas import DataFrame as df

# value to be tested
epochs = [10, 20, 30, 50, 80, 130, 210, 340, 550, 890]

# parameter control
alpha = 0.8
eta = 0.2
jumlah_neuron = (5,5)
k = 0

n_test = 10

qes = dict()


dataset_used = dataset.iloc[:, :].values
dataset_knn_ed = knn.impute_dataset(dataset_used, k)
dataset_norm = som.normalize_data(dataset_knn_ed)

for epoch in epochs:
    sum_all = 0
    qe_n = dict()
    for n in range(n_test):
        attr_size = len(dataset_used[0])
        print("Alpha: {0}, Eta: {1}, Epoch: {2}, Jumlah Neuron:{3}, K: {4} Try: {5}".format(alpha, eta, epoch, jumlah_neuron, k, n+1), end='\n')
        weight = som.init_som_net(jumlah_neuron[0], jumlah_neuron[1], attr_size)
        trained_weight = som.training(dataset_norm, weight, epoch, alpha, eta)
        qe = som.average_silhouette(trained_weight, dataset_norm)
        print("\nQE: {0}".format(qe), end='\n', flush=True)
        qe_n[n + 1] = qe
        sum_all += qe
    qe_n['avg'] = sum_all / n_test
    qes[str(epoch)] = qe_n

(df.from_dict(qes)).to_csv('silhouette_test/epoch_testing.csv')

Alpha: 0.8, Eta: 0.2, Epoch: 10, Jumlah Neuron:(5, 5), K: 0 Try: 1
Silhouette data 0 Cluster  4;2; : 0.33236338077480476
Silhouette data 3 Cluster  4;2; : 0.27981974396148096
Silhouette data 4 Cluster  4;2; : 0.3023070203976831
Silhouette data 5 Cluster  4;2; : 0.0031722250073055428
Silhouette data 6 Cluster  4;2; : 0.07992906503581389
Silhouette data 7 Cluster  4;2; : 0.38252390749241527
Silhouette data 10 Cluster  4;2; : 0.23014097323014301
Silhouette data 11 Cluster  4;2; : 0.4739183279145955
Silhouette data 12 Cluster  4;2; : 0.18121559825738678
Silhouette data 13 Cluster  4;2; : 0.3189882350430678
Silhouette data 14 Cluster  4;2; : 0.3020589902973757
Silhouette data 15 Cluster  4;2; : 0.24079542673562626
Silhouette data 16 Cluster  4;2; : 0.1946503061161801
Silhouette data 17 Cluster  4;2; : 0.1987246203981104
Silhouette data 18 Cluster  4;2; : 0.21174520669461203
Silhouette data 19 Cluster  4;2; : 0.3769245215368577
Silhouette data 20 Cluster  4;2; : 0.24595103713447775
Silhouett

Silhouette data 33 Cluster  3;1; : 0.4432493939181212
Silhouette data 34 Cluster  3;1; : 0.46404419461320723
Silhouette data 35 Cluster  3;1; : 0.4680252861084517
Silhouette data 36 Cluster  3;1; : 0.15493966487726826
Silhouette data 37 Cluster  3;1; : 0.4664497835940461
-------------------------------------------------------
Silhouette data 2 Cluster  1;3; : 0.06623637838396536
Silhouette data 9 Cluster  1;3; : 0.001031659145745163
-------------------------------------------------------
Silhouette data 5 Cluster  3;3; : 0.14897094636357466
Silhouette data 6 Cluster  3;3; : 0.08140582443898818
Silhouette data 12 Cluster  3;3; : -0.04404966811095602
Silhouette data 22 Cluster  3;3; : 0.02193224580582147
-------------------------------------------------------
Silhouette data 8 Cluster  0;4; : 0
-------------------------------------------------------
Silhouette data 24 Cluster  2;3; : 0
-------------------------------------------------------

QE: 0.21079357512976377
Alpha: 0.8, Eta: 0.2, 

Alpha: 0.8, Eta: 0.2, Epoch: 10, Jumlah Neuron:(5, 5), K: 0 Try: 8
Silhouette data 0 Cluster  2;0; : 0.3399355555617363
Silhouette data 1 Cluster  2;0; : 0.09252384335609139
Silhouette data 3 Cluster  2;0; : 0.2631129453421493
Silhouette data 4 Cluster  2;0; : 0.24789226860212782
Silhouette data 5 Cluster  2;0; : -0.04926980429209101
Silhouette data 7 Cluster  2;0; : 0.3673847105716096
Silhouette data 9 Cluster  2;0; : -0.043572441572231446
Silhouette data 10 Cluster  2;0; : 0.22167134841524389
Silhouette data 11 Cluster  2;0; : 0.44639341901153073
Silhouette data 12 Cluster  2;0; : 0.07820078783536898
Silhouette data 13 Cluster  2;0; : 0.29195718329477044
Silhouette data 14 Cluster  2;0; : 0.2140867102037529
Silhouette data 15 Cluster  2;0; : 0.17711899212428817
Silhouette data 16 Cluster  2;0; : 0.24740448994310804
Silhouette data 17 Cluster  2;0; : 0.1471152014536559
Silhouette data 18 Cluster  2;0; : 0.19747620898776674
Silhouette data 19 Cluster  2;0; : 0.3628194070714831
Silhouet

Alpha: 0.8, Eta: 0.2, Epoch: 20, Jumlah Neuron:(5, 5), K: 0 Try: 2
Silhouette data 0 Cluster  4;4; : 0.320814860757388
Silhouette data 3 Cluster  4;4; : 0.33757648429029125
Silhouette data 4 Cluster  4;4; : 0.3209443521803027
Silhouette data 5 Cluster  4;4; : -0.0004669180045412893
Silhouette data 6 Cluster  4;4; : 0.159839797820807
Silhouette data 7 Cluster  4;4; : 0.38330412555509
Silhouette data 9 Cluster  4;4; : 0.07475102761355339
Silhouette data 10 Cluster  4;4; : 0.26084452070869224
Silhouette data 11 Cluster  4;4; : 0.4699911784834003
Silhouette data 12 Cluster  4;4; : 0.20847338101374022
Silhouette data 13 Cluster  4;4; : 0.338687208426119
Silhouette data 14 Cluster  4;4; : 0.32998539549518685
Silhouette data 15 Cluster  4;4; : 0.20901362965648823
Silhouette data 16 Cluster  4;4; : 0.1856417641414912
Silhouette data 17 Cluster  4;4; : 0.19355571982857492
Silhouette data 18 Cluster  4;4; : 0.268722783652589
Silhouette data 19 Cluster  4;4; : 0.37577310764652105
Silhouette data 

Silhouette data 34 Cluster  1;3; : 0.4650057500929094
Silhouette data 35 Cluster  1;3; : 0.4661451054781544
Silhouette data 36 Cluster  1;3; : 0.2765238922467372
Silhouette data 37 Cluster  1;3; : 0.4637031767755526
-------------------------------------------------------
Silhouette data 1 Cluster  2;4; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  4;3; : 0
-------------------------------------------------------
Silhouette data 8 Cluster  4;4; : 0
-------------------------------------------------------
Silhouette data 24 Cluster  4;1; : 0
-------------------------------------------------------

QE: 0.25575735870075356
Alpha: 0.8, Eta: 0.2, Epoch: 20, Jumlah Neuron:(5, 5), K: 0 Try: 6
Silhouette data 0 Cluster  2;1; : 0.35214706484544905
Silhouette data 3 Cluster  2;1; : 0.31421443511483643
Silhouette data 4 Cluster  2;1; : 0.06920724013479958
Silhouette data 7 Cluster  2;1; : 0.31974379936862946
Silhouette data 10 Cluster  2;1; : 0.19886786935948

...

Alpha: 0.8, Eta: 0.2, Epoch: 550, Jumlah Neuron:(5, 5), K: 0 Try: 2
Silhouette data 0 Cluster  4;3; : 0.3555830683953976
Silhouette data 3 Cluster  4;3; : 0.3074974279994521
Silhouette data 4 Cluster  4;3; : 0.09550726563540221
Silhouette data 7 Cluster  4;3; : 0.29440608365318516
Silhouette data 10 Cluster  4;3; : 0.20142102467546802
Silhouette data 11 Cluster  4;3; : 0.482647598297766
Silhouette data 12 Cluster  4;3; : 0.05276771421135978
Silhouette data 13 Cluster  4;3; : 0.22365202472988044
Silhouette data 14 Cluster  4;3; : 0.176410396763509
Silhouette data 15 Cluster  4;3; : -0.03213716954830564
Silhouette data 16 Cluster  4;3; : 0.04164248283676243
Silhouette data 17 Cluster  4;3; : -0.03711743155023118
Silhouette data 18 Cluster  4;3; : 0.20492833782973804
Silhouette data 19 Cluster  4;3; : 0.3123100169175591
Silhouette data 20 Cluster  4;3; : 0.09007703490562621
Silhouette data 21 Cluster  4;3; : 0.061980500871287085
Silhouette data 23 Cluster  4;3; : 0.3168695429129031
Silhou

Alpha: 0.8, Eta: 0.2, Epoch: 550, Jumlah Neuron:(5, 5), K: 0 Try: 6
Silhouette data 0 Cluster  3;1; : 0.3708491734530503
Silhouette data 1 Cluster  3;1; : 0.08861052427526744
Silhouette data 3 Cluster  3;1; : 0.2518449518065603
Silhouette data 4 Cluster  3;1; : 0.3110321141256712
Silhouette data 5 Cluster  3;1; : -0.0068153900150622285
Silhouette data 6 Cluster  3;1; : 0.12169248035261232
Silhouette data 7 Cluster  3;1; : 0.35684805565027267
Silhouette data 9 Cluster  3;1; : -0.042809445278457585
Silhouette data 10 Cluster  3;1; : 0.21395637733223757
Silhouette data 11 Cluster  3;1; : 0.44858315519048075
Silhouette data 12 Cluster  3;1; : 0.1708890276624354
Silhouette data 13 Cluster  3;1; : 0.2973137186085143
Silhouette data 14 Cluster  3;1; : 0.2826460903506757
Silhouette data 15 Cluster  3;1; : 0.17159199107792342
Silhouette data 16 Cluster  3;1; : 0.23935048530757783
Silhouette data 17 Cluster  3;1; : 0.21218507804061099
Silhouette data 18 Cluster  3;1; : 0.18873378563372661
Silhou

Silhouette data 1 Cluster  4;3; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  1;2; : 0.06623637838396536
Silhouette data 9 Cluster  1;2; : 0.038914116055660926
-------------------------------------------------------
Silhouette data 8 Cluster  3;0; : 0
-------------------------------------------------------
Silhouette data 24 Cluster  4;2; : 0
-------------------------------------------------------

QE: 0.26523147091645854
Alpha: 0.8, Eta: 0.2, Epoch: 550, Jumlah Neuron:(5, 5), K: 0 Try: 10
Silhouette data 0 Cluster  4;3; : 0.2680480574352812
Silhouette data 3 Cluster  4;3; : 0.16606456153347965
Silhouette data 7 Cluster  4;3; : 0.05832715925173868
Silhouette data 10 Cluster  4;3; : 0.09866667128901116
Silhouette data 11 Cluster  4;3; : 0.43393322280379115
Silhouette data 13 Cluster  4;3; : 0.08673205695399998
Silhouette data 14 Cluster  4;3; : 0.08155400685158731
Silhouette data 19 Cluster  4;3; : 0.07415594744253745
Silhouette data 20 Cluster  

Alpha: 0.8, Eta: 0.2, Epoch: 890, Jumlah Neuron:(5, 5), K: 0 Try: 4
Silhouette data 0 Cluster  4;1; : 0.2916258040088998
Silhouette data 3 Cluster  4;1; : 0.21871745967814474
Silhouette data 7 Cluster  4;1; : 0.137328045397545
Silhouette data 10 Cluster  4;1; : 0.1333543809357695
Silhouette data 11 Cluster  4;1; : 0.4670965146091539
Silhouette data 13 Cluster  4;1; : 0.14964020996164498
Silhouette data 14 Cluster  4;1; : 0.133007806258226
Silhouette data 19 Cluster  4;1; : 0.13815386266440993
Silhouette data 23 Cluster  4;1; : 0.2521192424016368
Silhouette data 25 Cluster  4;1; : 0.453012431298961
Silhouette data 26 Cluster  4;1; : 0.3013831535141686
Silhouette data 27 Cluster  4;1; : 0.050931601471744345
Silhouette data 28 Cluster  4;1; : 0.15229973771941305
Silhouette data 29 Cluster  4;1; : 0.3420339007148616
Silhouette data 30 Cluster  4;1; : 0.4123614639808679
Silhouette data 31 Cluster  4;1; : 0.23798185557365828
Silhouette data 32 Cluster  4;1; : 0.23506068851284875
Silhouette d

Alpha: 0.8, Eta: 0.2, Epoch: 890, Jumlah Neuron:(5, 5), K: 0 Try: 8
Silhouette data 0 Cluster  2;3; : 0.33236338077480476
Silhouette data 3 Cluster  2;3; : 0.27981974396148096
Silhouette data 4 Cluster  2;3; : 0.3023070203976831
Silhouette data 5 Cluster  2;3; : 0.0031722250073055428
Silhouette data 6 Cluster  2;3; : 0.07992906503581389
Silhouette data 7 Cluster  2;3; : 0.38252390749241527
Silhouette data 10 Cluster  2;3; : 0.23014097323014301
Silhouette data 11 Cluster  2;3; : 0.4739183279145955
Silhouette data 12 Cluster  2;3; : 0.18121559825738678
Silhouette data 13 Cluster  2;3; : 0.3189882350430678
Silhouette data 14 Cluster  2;3; : 0.3020589902973757
Silhouette data 15 Cluster  2;3; : 0.24079542673562626
Silhouette data 16 Cluster  2;3; : 0.1946503061161801
Silhouette data 17 Cluster  2;3; : 0.1987246203981104
Silhouette data 18 Cluster  2;3; : 0.21174520669461203
Silhouette data 19 Cluster  2;3; : 0.3769245215368577
Silhouette data 20 Cluster  2;3; : 0.24595103713447775
Silhouet

# Pengujian parameter nilai jumlah neuron

Nilai yang akan diujikan
- 2x2, 3x3, 4x4, 5x5

Variabel kontrol
- Nilai alpha = 1
- Nilai eta = 0.7
- Jumlah epoch = 600
- Nilai K = 0

In [15]:
from IPython.display import clear_output
from time import sleep
from pandas import DataFrame as df
import json

# value to be tested
eta = 0.2

# parameter control
alpha = 0.8
epoch = 233
jumlah_neurons = [(2,2), (3,3), (4,4), (5,5)]
k = 0

n_test = 10

qes = dict()


dataset_used = dataset.iloc[:, :].values
dataset_knn_ed = knn.impute_dataset(dataset_used, k)
dataset_norm = som.normalize_data(dataset_knn_ed)

def cluster_build(dataset_indexes, dataset_input, trained_weight):
    cluster_result = dict()
    cls_list = list()
    for idx, x_data in enumerate(dataset_input):
        indexes = dataset_indexes[idx]
        c = som.penentuan_cluster(trained_weight, x_data)
        cls_id = str(c[0]) + ';' + str(c[1]) + ';'
        if cls_id not in cls_list:
            cls_list.append(cls_id)
            cluster_result[cls_id] = list()
        cluster_result[cls_id].append(indexes)
    return cluster_result
    

for jumlah_neuron in jumlah_neurons:
    sum_all = 0
    qe_n = dict()
    max_qe = -999999
    for n in range(n_test):
        attr_size = len(dataset_used[0])
        print("Alpha: {0}, Eta: {1}, Epoch: {2}, Jumlah Neuron:{3}, K: {4} Try: {5}".format(alpha, eta, epoch, jumlah_neuron, k, n+1), end='\n')
        weight = som.init_som_net(jumlah_neuron[0], jumlah_neuron[1], attr_size)
        trained_weight = som.training(dataset_norm, weight, epoch, alpha, eta)
        qe = som.average_silhouette(trained_weight, dataset_norm)
        print("\nQE: {0}".format(qe), end='\n', flush=True)
        qe_n[n + 1] = qe
        sum_all += qe
        
        if qe > max_qe:
            max_qe = qe
            cls_result = cluster_build(dataset.index, dataset_norm, trained_weight)
            cls_result['score'] = max_qe
            with open('neuron_test_result/' + str(jumlah_neuron) + 'sil.json', 'w') as file:
                file.write(json.dumps(cls_result))
    qe_n['avg'] = sum_all / n_test
    qes[str(jumlah_neuron)] = qe_n

(df.from_dict(qes)).to_csv('silhouette_test/jumlah_neuron_testing.csv')

Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(2, 2), K: 0 Try: 1
Silhouette data 0 Cluster  0;1; : 0.5138943066870314
Silhouette data 1 Cluster  0;1; : 0.1680393858203587
Silhouette data 2 Cluster  0;1; : 0.2713926522605402
Silhouette data 3 Cluster  0;1; : 0.4379059779502703
Silhouette data 4 Cluster  0;1; : 0.43067080937154006
Silhouette data 5 Cluster  0;1; : -0.013898023328504358
Silhouette data 6 Cluster  0;1; : 0.2454789893442351
Silhouette data 7 Cluster  0;1; : 0.489383516502734
Silhouette data 9 Cluster  0;1; : 0.10793501725856186
Silhouette data 10 Cluster  0;1; : 0.29384227840421434
Silhouette data 11 Cluster  0;1; : 0.5818502584183236
Silhouette data 12 Cluster  0;1; : 0.23492953921484078
Silhouette data 13 Cluster  0;1; : 0.4134575067263591
Silhouette data 14 Cluster  0;1; : 0.4195747751990124
Silhouette data 15 Cluster  0;1; : 0.3260087032827146
Silhouette data 16 Cluster  0;1; : 0.3387959503466058
Silhouette data 17 Cluster  0;1; : 0.32760830479853653
Silhouette data 

Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(2, 2), K: 0 Try: 5
Silhouette data 0 Cluster  1;0; : 0.5138943066870314
Silhouette data 1 Cluster  1;0; : 0.1680393858203587
Silhouette data 2 Cluster  1;0; : 0.2713926522605402
Silhouette data 3 Cluster  1;0; : 0.4379059779502703
Silhouette data 4 Cluster  1;0; : 0.43067080937154006
Silhouette data 5 Cluster  1;0; : -0.013898023328504358
Silhouette data 6 Cluster  1;0; : 0.2454789893442351
Silhouette data 7 Cluster  1;0; : 0.489383516502734
Silhouette data 9 Cluster  1;0; : 0.10793501725856186
Silhouette data 10 Cluster  1;0; : 0.29384227840421434
Silhouette data 11 Cluster  1;0; : 0.5818502584183236
Silhouette data 12 Cluster  1;0; : 0.23492953921484078
Silhouette data 13 Cluster  1;0; : 0.4134575067263591
Silhouette data 14 Cluster  1;0; : 0.4195747751990124
Silhouette data 15 Cluster  1;0; : 0.3260087032827146
Silhouette data 16 Cluster  1;0; : 0.3387959503466058
Silhouette data 17 Cluster  1;0; : 0.32760830479853653
Silhouette data 

-------------------------------------------------------
Silhouette data 5 Cluster  0;1; : 0.11142260971809141
Silhouette data 8 Cluster  0;1; : 0.2554618987522899
Silhouette data 22 Cluster  0;1; : 0.010577800325293317
Silhouette data 24 Cluster  0;1; : 0.0023742974417389016
-------------------------------------------------------

QE: 0.2573185205081599
Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(2, 2), K: 0 Try: 9
Silhouette data 0 Cluster  1;0; : 0.5138943066870314
Silhouette data 1 Cluster  1;0; : 0.1680393858203587
Silhouette data 2 Cluster  1;0; : 0.2713926522605402
Silhouette data 3 Cluster  1;0; : 0.4379059779502703
Silhouette data 4 Cluster  1;0; : 0.43067080937154006
Silhouette data 5 Cluster  1;0; : -0.013898023328504358
Silhouette data 6 Cluster  1;0; : 0.2454789893442351
Silhouette data 7 Cluster  1;0; : 0.489383516502734
Silhouette data 9 Cluster  1;0; : 0.10793501725856186
Silhouette data 10 Cluster  1;0; : 0.29384227840421434
Silhouette data 11 Cluster  1;0; : 0.581

Silhouette data 18 Cluster  1;0; : 0.2008800911986188
Silhouette data 19 Cluster  1;0; : 0.364162757828315
Silhouette data 20 Cluster  1;0; : 0.23630511505748225
Silhouette data 21 Cluster  1;0; : 0.28538583482533625
Silhouette data 22 Cluster  1;0; : 0.10976867719242613
Silhouette data 23 Cluster  1;0; : 0.332432412970326
Silhouette data 25 Cluster  1;0; : 0.46444921702042347
Silhouette data 26 Cluster  1;0; : 0.38431525073253137
Silhouette data 27 Cluster  1;0; : 0.20558062925183443
Silhouette data 28 Cluster  1;0; : 0.19401676540504822
Silhouette data 29 Cluster  1;0; : 0.37954989091230396
Silhouette data 30 Cluster  1;0; : 0.4534568301926958
Silhouette data 31 Cluster  1;0; : 0.3079780203682401
Silhouette data 32 Cluster  1;0; : 0.3728295395542971
Silhouette data 33 Cluster  1;0; : 0.4452443204409948
Silhouette data 34 Cluster  1;0; : 0.45688947559077914
Silhouette data 35 Cluster  1;0; : 0.4631591396732766
Silhouette data 36 Cluster  1;0; : 0.21185913334120518
Silhouette data 37 C

Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(3, 3), K: 0 Try: 6
Silhouette data 0 Cluster  0;2; : 0.320814860757388
Silhouette data 3 Cluster  0;2; : 0.33757648429029125
Silhouette data 4 Cluster  0;2; : 0.3209443521803027
Silhouette data 5 Cluster  0;2; : -0.0004669180045412893
Silhouette data 6 Cluster  0;2; : 0.159839797820807
Silhouette data 7 Cluster  0;2; : 0.38330412555509
Silhouette data 9 Cluster  0;2; : 0.07475102761355339
Silhouette data 10 Cluster  0;2; : 0.26084452070869224
Silhouette data 11 Cluster  0;2; : 0.4699911784834003
Silhouette data 12 Cluster  0;2; : 0.20847338101374022
Silhouette data 13 Cluster  0;2; : 0.338687208426119
Silhouette data 14 Cluster  0;2; : 0.32998539549518685
Silhouette data 15 Cluster  0;2; : 0.20901362965648823
Silhouette data 16 Cluster  0;2; : 0.1856417641414912
Silhouette data 17 Cluster  0;2; : 0.19355571982857492
Silhouette data 18 Cluster  0;2; : 0.268722783652589
Silhouette data 19 Cluster  0;2; : 0.37577310764652105
Silhouette data

...

Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(4, 4), K: 0 Try: 4
Silhouette data 0 Cluster  1;1; : 0.35210511340582584
Silhouette data 3 Cluster  1;1; : 0.3345356106457587
Silhouette data 4 Cluster  1;1; : 0.25251297869690714
Silhouette data 6 Cluster  1;1; : 0.0733501527453783
Silhouette data 7 Cluster  1;1; : 0.3650764229006815
Silhouette data 10 Cluster  1;1; : 0.22246933427130483
Silhouette data 11 Cluster  1;1; : 0.499705314717968
Silhouette data 12 Cluster  1;1; : 0.1358610588630306
Silhouette data 13 Cluster  1;1; : 0.29314024313041437
Silhouette data 14 Cluster  1;1; : 0.28238949313421524
Silhouette data 15 Cluster  1;1; : 0.10615266627841291
Silhouette data 16 Cluster  1;1; : 0.13272058227815087
Silhouette data 17 Cluster  1;1; : 0.1094723552041235
Silhouette data 18 Cluster  1;1; : 0.2632089557166484
Silhouette data 19 Cluster  1;1; : 0.3720160369203642
Silhouette data 20 Cluster  1;1; : 0.23136327418733693
Silhouette data 21 Cluster  1;1; : 0.19473323570662884
Silhouette 

Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(4, 4), K: 0 Try: 8
Silhouette data 0 Cluster  0;2; : 0.3675918199690218
Silhouette data 1 Cluster  0;2; : 0.09106641765034457
Silhouette data 3 Cluster  0;2; : 0.26452965759769564
Silhouette data 4 Cluster  0;2; : 0.29195777188232586
Silhouette data 5 Cluster  0;2; : -0.0035144743036891976
Silhouette data 6 Cluster  0;2; : 0.0728913511688957
Silhouette data 7 Cluster  0;2; : 0.37019963757117613
Silhouette data 10 Cluster  0;2; : 0.2194548284889243
Silhouette data 11 Cluster  0;2; : 0.4581637568540137
Silhouette data 12 Cluster  0;2; : 0.17452932542309196
Silhouette data 13 Cluster  0;2; : 0.306562743179402
Silhouette data 14 Cluster  0;2; : 0.2903579284453785
Silhouette data 15 Cluster  0;2; : 0.23065550333112372
Silhouette data 16 Cluster  0;2; : 0.24730205494545768
Silhouette data 17 Cluster  0;2; : 0.20425531965461818
Silhouette data 18 Cluster  0;2; : 0.2008800911986188
Silhouette data 19 Cluster  0;2; : 0.364162757828315
Silhouette 

Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(5, 5), K: 0 Try: 2
Silhouette data 0 Cluster  4;2; : 0.33236338077480476
Silhouette data 3 Cluster  4;2; : 0.27981974396148096
Silhouette data 4 Cluster  4;2; : 0.3023070203976831
Silhouette data 5 Cluster  4;2; : 0.0031722250073055428
Silhouette data 6 Cluster  4;2; : 0.07992906503581389
Silhouette data 7 Cluster  4;2; : 0.38252390749241527
Silhouette data 10 Cluster  4;2; : 0.23014097323014301
Silhouette data 11 Cluster  4;2; : 0.4739183279145955
Silhouette data 12 Cluster  4;2; : 0.18121559825738678
Silhouette data 13 Cluster  4;2; : 0.3189882350430678
Silhouette data 14 Cluster  4;2; : 0.3020589902973757
Silhouette data 15 Cluster  4;2; : 0.24079542673562626
Silhouette data 16 Cluster  4;2; : 0.1946503061161801
Silhouette data 17 Cluster  4;2; : 0.1987246203981104
Silhouette data 18 Cluster  4;2; : 0.21174520669461203
Silhouette data 19 Cluster  4;2; : 0.3769245215368577
Silhouette data 20 Cluster  4;2; : 0.24595103713447775
Silhouet

Silhouette data 22 Cluster  0;0; : 0.11485990181739039
Silhouette data 23 Cluster  0;0; : 0.34255508111619126
Silhouette data 25 Cluster  0;0; : 0.4796981171997888
Silhouette data 26 Cluster  0;0; : 0.38624680729227184
Silhouette data 27 Cluster  0;0; : 0.11330173066849693
Silhouette data 28 Cluster  0;0; : 0.20100886930501036
Silhouette data 29 Cluster  0;0; : 0.39360484213958297
Silhouette data 30 Cluster  0;0; : 0.4692547759344028
Silhouette data 31 Cluster  0;0; : 0.3190738546952169
Silhouette data 32 Cluster  0;0; : 0.3855534335878613
Silhouette data 33 Cluster  0;0; : 0.4616229301796726
Silhouette data 34 Cluster  0;0; : 0.47268128427777073
Silhouette data 35 Cluster  0;0; : 0.479322513425012
Silhouette data 36 Cluster  0;0; : 0.22291697747069214
Silhouette data 37 Cluster  0;0; : 0.4794027521470296
-------------------------------------------------------
Silhouette data 1 Cluster  3;1; : 0
-------------------------------------------------------
Silhouette data 2 Cluster  4;1; : 0

Alpha: 0.8, Eta: 0.2, Epoch: 233, Jumlah Neuron:(5, 5), K: 0 Try: 9
Silhouette data 0 Cluster  0;1; : 0.3675918199690218
Silhouette data 1 Cluster  0;1; : 0.09106641765034457
Silhouette data 3 Cluster  0;1; : 0.26452965759769564
Silhouette data 4 Cluster  0;1; : 0.29195777188232586
Silhouette data 5 Cluster  0;1; : -0.0035144743036891976
Silhouette data 6 Cluster  0;1; : 0.0728913511688957
Silhouette data 7 Cluster  0;1; : 0.37019963757117613
Silhouette data 10 Cluster  0;1; : 0.2194548284889243
Silhouette data 11 Cluster  0;1; : 0.4581637568540137
Silhouette data 12 Cluster  0;1; : 0.17452932542309196
Silhouette data 13 Cluster  0;1; : 0.306562743179402
Silhouette data 14 Cluster  0;1; : 0.2903579284453785
Silhouette data 15 Cluster  0;1; : 0.23065550333112372
Silhouette data 16 Cluster  0;1; : 0.24730205494545768
Silhouette data 17 Cluster  0;1; : 0.20425531965461818
Silhouette data 18 Cluster  0;1; : 0.2008800911986188
Silhouette data 19 Cluster  0;1; : 0.364162757828315
Silhouette 

# Pengujian parameter nilai K

Nilai yang akan diujikan
- 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1

Variabel kontrol
- Nilai eta = 0.5
- Jumlah epoch = 200
- Jumlah neuron = 5x5
- Nilai K = 0

In [8]:
from IPython.display import clear_output
from time import sleep
from pandas import DataFrame as df

# value to be tested
epochs = [13, 21, 34, 55, 89, 144, 233, 377, 610, 987]

# parameter control
alpha = 0.1
eta = 0.7
jumlah_neuron = (5,5)
k = 0

n_test = 10

qes = dict()


dataset_used = dataset.iloc[:, :].values
dataset_knn_ed = knn.impute_dataset(dataset_used, k)
dataset_norm = som.normalize_data(dataset_knn_ed)

for epoch in epochs:
    sum_all = 0
    qe_n = dict()
    for n in range(n_test):
        attr_size = len(dataset_used[0])
        print("Alpha: {0}, Eta: {1}, Epoch: {2}, Jumlah Neuron:{3}, K: {4} Try: {5}".format(alpha, eta, epoch, jumlah_neuron, k, n+1), end='\n')
        weight = som.init_som_net(jumlah_neuron[0], jumlah_neuron[1], attr_size)
        trained_weight = som.training(dataset_norm, weight, epoch, alpha, eta)
        qe = som.quantization_error(trained_weight, dataset_norm)
        print("\nQE: {0}".format(qe), end='\n', flush=True)
        qe_n[n + 1] = qe
        sum_all += qe
            
    qe_n['avg'] = sum_all / n_test
    qes[str(epoch)] = qe_n

# (df.from_dict(qes)).to_csv('epoch_testing_2.csv')

Alpha: 0.1, Eta: 0.7, Epoch: 13, Jumlah Neuron:(5, 5), K: 0 Try: 1
Progress: 100.0% QE:0.87676488336809931066256917796
QE: 0.8767648833680993
Alpha: 0.1, Eta: 0.7, Epoch: 13, Jumlah Neuron:(5, 5), K: 0 Try: 2
Progress: 100.0% QE:0.89362232717339338015853692996
QE: 0.8936223271733933
Alpha: 0.1, Eta: 0.7, Epoch: 13, Jumlah Neuron:(5, 5), K: 0 Try: 3
Progress: 100.0% QE:0.91335529572503729764316182574
QE: 0.9133552957250372
Alpha: 0.1, Eta: 0.7, Epoch: 13, Jumlah Neuron:(5, 5), K: 0 Try: 4
Progress: 100.0% QE:0.87510893367342287962502927417
QE: 0.8751089336734228
Alpha: 0.1, Eta: 0.7, Epoch: 13, Jumlah Neuron:(5, 5), K: 0 Try: 5
Progress: 100.0% QE:0.89055403773883690719138169956
QE: 0.8905540377388369
Alpha: 0.1, Eta: 0.7, Epoch: 13, Jumlah Neuron:(5, 5), K: 0 Try: 6
Progress: 100.0% QE:0.88809695474530346872510959477
QE: 0.8880969547453034
Alpha: 0.1, Eta: 0.7, Epoch: 13, Jumlah Neuron:(5, 5), K: 0 Try: 7
Progress: 100.0% QE:0.92687966481048624093803536175
QE: 0.9268796648104862
Alpha:

Progress: 100.0% QE:0.79055996379370567274568707444
QE: 0.7905599637937056
Alpha: 0.1, Eta: 0.7, Epoch: 144, Jumlah Neuron:(5, 5), K: 0 Try: 9
Progress: 100.0% QE:0.83254886187041815798100203042
QE: 0.8325488618704181
Alpha: 0.1, Eta: 0.7, Epoch: 144, Jumlah Neuron:(5, 5), K: 0 Try: 10
Progress: 100.0% QE:0.82205774911418877874315387452
QE: 0.8220577491141887
Alpha: 0.1, Eta: 0.7, Epoch: 233, Jumlah Neuron:(5, 5), K: 0 Try: 1
Progress: 100.0% QE:0.80247128727741133333011785568
QE: 0.8024712872774113
Alpha: 0.1, Eta: 0.7, Epoch: 233, Jumlah Neuron:(5, 5), K: 0 Try: 2
Progress: 100.0% QE:0.81025068807211020031483592098
QE: 0.8102506880721102
Alpha: 0.1, Eta: 0.7, Epoch: 233, Jumlah Neuron:(5, 5), K: 0 Try: 3
Progress: 100.0% QE:0.77920289111354764538978061174
QE: 0.7792028911135476
Alpha: 0.1, Eta: 0.7, Epoch: 233, Jumlah Neuron:(5, 5), K: 0 Try: 4
Progress: 100.0% QE:0.78900434557835765787081624638
QE: 0.7890043455783576
Alpha: 0.1, Eta: 0.7, Epoch: 233, Jumlah Neuron:(5, 5), K: 0 Try: 

In [9]:
a = dict()
len(a)

0